<a href="https://colab.research.google.com/github/Umerfarooq122/conditional-multinomial-logit-model-in-python/blob/main/Blog_4_Conditional_multinomial_logit_model_in_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Conditional Multinomial Logit Models In Python**

The conditional multinomial logit (cMNL) model is often suitable for scenarios where you have discrete choice data and observations can be grouped or clustered. It is commonly used in the analysis of panel or grouped data, where individuals or entities make choices within specific groups or clusters

## **Importing Libraries and Loading Dataset:**

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/Umerfarooq122/conditional-multinomial-logit-model-in-python/main/healthcare_dataset.csv')

In [ ]:
data.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Tiffany Ramirez,81,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,37490.983364,146,Elective,2022-12-01,Aspirin,Inconclusive
1,Ruben Burns,35,Male,O+,Asthma,2023-06-01,Diane Jackson,"Burke, Griffin and Cooper",UnitedHealthcare,47304.064845,404,Emergency,2023-06-15,Lipitor,Normal
2,Chad Byrd,61,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,36874.896997,292,Emergency,2019-02-08,Lipitor,Normal
3,Antonio Frederick,49,Male,B-,Asthma,2020-05-02,Brian Chandler,Garcia Ltd,Medicare,23303.322092,480,Urgent,2020-05-03,Penicillin,Abnormal
4,Mrs. Brandy Flowers,51,Male,O-,Arthritis,2021-07-09,Dustin Griffin,"Jones, Brown and Murray",UnitedHealthcare,18086.344184,477,Urgent,2021-08-02,Paracetamol,Normal


## **Preprocessing the Data:**

In [ ]:
data.isna().sum()

Name                  0
Age                   0
Gender                0
Blood Type            0
Medical Condition     0
Date of Admission     0
Doctor                0
Hospital              0
Insurance Provider    0
Billing Amount        0
Room Number           0
Admission Type        0
Discharge Date        0
Medication            0
Test Results          0
dtype: int64

In [ ]:
data = data[['Age','Gender','Blood Type','Medical Condition','Test Results']]

In [ ]:
data.head()

,Age,Gender,Blood Type,Medical Condition,Test Results
0,81,Female,O-,Diabetes,Inconclusive
1,35,Male,O+,Asthma,Normal
2,61,Male,B-,Obesity,Normal
3,49,Male,B-,Asthma,Abnormal
4,51,Male,O-,Arthritis,Normal


In [ ]:
data.shape

(10000, 5)

In [ ]:
dummy1 = pd.get_dummies(data.Gender)

In [ ]:
dummy2 = pd.get_dummies(data['Blood Type'])

In [ ]:
dummy2

,A+,A-,AB+,AB-,B+,B-,O+,O-
0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
9995,1,0,0,0,0,0,0,0
9996,0,0,1,0,0,0,0,0
9997,0,0,0,0,0,1,0,0
9998,1,0,0,0,0,0,0,0


In [ ]:
dummy3 = pd.get_dummies(data['Medical Condition'])

In [ ]:
dummy4  = data['Test Results'].replace(['Inconclusive', 'Normal','Abnormal'],
                        ['0', '1','2'], inplace=True)

In [ ]:
final = pd.concat([data, dummy1], axis='columns')
final = pd.concat([final, dummy2], axis='columns')
final = pd.concat([final, dummy3], axis='columns')
final = pd.concat([final, dummy4], axis='columns')

In [ ]:
final.head()

,Age,Gender,Blood Type,Medical Condition,Test Results,Female,Male,A+,A-,AB+,...,B+,B-,O+,O-,Arthritis,Asthma,Cancer,Diabetes,Hypertension,Obesity
0,81,Female,O-,Diabetes,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
1,35,Male,O+,Asthma,1,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
2,61,Male,B-,Obesity,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,49,Male,B-,Asthma,2,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
4,51,Male,O-,Arthritis,1,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,0
